In [1]:
# Author: Arthur Prigent
# Email: aprigent@geomar.de

In [3]:
import xarray as xr 
#import xgcm 
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
import glob
import cartopy.crs as ccrs
import warnings
warnings.filterwarnings('ignore')
import matplotlib.ticker as mticker
import cartopy
from scipy import stats
import pandas as pd
from datetime import datetime
now = datetime.now()
def data_sub(data,lon_min,lon_max,lat_min,lat_max):
    
    '''Define a box between lon_min lon_max lat_min and lat_max and 
    extract the data in the box and drop everything else.
    
    
    Parameters
    ----------
    
    data : xarray_like
    Data to be subdomained. 
    
    lon_min : integer
    Longitude minimum of the subdomain
    
    lon_max : integer
    Longitude maximum of the subdomain
    
    lat_min : integer
    Latitude minimum of the subdomain
    
    lat_max : integer
    Latitude maximum of the subdomain
    
    Returns
    ---------
    
    data_sub : xarray_like
    Subdomain. 
    '''
    
    try:
        data_sub = data.where((  data.lon>=lon_min) & (data.lon<=lon_max) & (data.lat<=lat_max) & (data.lat>=lat_min),
                                                                          drop=True)
    except AttributeError:
        try:
            data_sub = data.where((  data.nav_lon>=lon_min) & (data.nav_lon<=lon_max) & (data.nav_lat<=lat_max) & (data.nav_lat>=lat_min),drop=True)
        except AttributeError:
            try:
                data_sub = data.where((  data.longitude>=lon_min) & (data.longitude<=lon_max) & (data.latitude<=lat_max) & (data.latitude>=lat_min),drop=True)
            except AttributeError:
                try:
                    data_sub = data.where((  data.x>=lon_min) & (data.x<=lon_max) & (data.y<=lat_max) &
                                      (data.y>=lat_min),drop=True)
                except AttributeError:
                    data_sub = data.where((  data.LON>=lon_min) & (data.LON<=lon_max) & (data.LAT<=lat_max) &
                                      (data.LAT>=lat_min),drop=True)
            
    

 
    
    return data_sub
import scipy.optimize as sc_o


def nandetrend(y):
    ''' Remove the linear trend from the data '''
    
    x = np.arange(0,y.shape[0],1)
    m, b, r_val, p_val, std_err = stats.linregress(x,np.array(y))
    y_detrended= np.array(y) - m*x -b
    return y_detrended

dir_cmip5 = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip5_data/'
dir_cmip6 = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip6_data/'


data_cmip_hist_ts = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip6_data/historical/ts/'
data_cmip_ssp245_ts = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip6_data/ssp245/ts/'
data_cmip_ssp585_ts = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip6_data/ssp585/ts/'


path_fig='/data/user/aprigent/projects/uncertainty_ATL3_ABA/figures/'
path_data_out = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/'

# List of models and scenario considered

In [4]:


model_list_CMIP6=['ACCESS-CM2','ACCESS-ESM1-5','AWI-CM-1-1-MR','BCC-CSM2-MR','CAMS-CSM1-0',
            'CESM2-WACCM','FGOALS-f3-L','FGOALS-g3','GFDL-CM4','GFDL-ESM4','INM-CM4-8',
            'INM-CM5-0','IPSL-CM6A-LR','MIROC6','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3',
            'NorESM2-LM','NorESM2-MM']

scenario_CMIP6=['ssp245','ssp585']
len(model_list_CMIP6)

20

# Functions to get the ATL3 SST variability

In [5]:
def make_sst_atl3(sst_tmp):

    sst_tmp = sst_tmp[:] - 273.15 # convert to degree C
    sst_tmp = xr.concat([sst_tmp[ :,:, 72:], sst_tmp[:, :, :72]], dim='lon')
    sst_tmp.coords['lon'] = (sst_tmp.coords['lon'] + 180) % 360 - 180
    sst_atl3 = data_sub(sst_tmp,-20,0,-3,3)
    sst_atl3 = sst_atl3.mean(dim='lon').mean(dim='lat')

    return sst_atl3
def ano_norm_t(ds):
    
    '''Compute the anomalies by removing the monthly means. 
    The anomalies are normalized by their corresponding month.
    
    Parameters
    ----------
    
    ds : xarray_like
    Timeserie or 3d field.
    
    Returns
    -----------
    
    ano : xarray_like
    Returns the anomalies of var relative the climatology.
    
    ano_norm : xarray_like
    Returns the anomalies of var relative the climatology normalized by the standard deviation.
    
    '''    
    
    clim     = ds.groupby('time.month').mean('time')
    clim_std = ds.groupby('time.month').std('time')
    ano      = ds.groupby('time.month') - clim
    ano_norm = xr.apply_ufunc(lambda x, m, s: (x - m) / s,
                                    ds.groupby('time.month'),
                                    clim, clim_std)
    
    return ano, ano_norm


def compute_atl3_amplitude(sst_atl3,period_str,period_end):
    
    try:

        sst_atl3 = sst_atl3.sel(time=slice(datetime(period_str, 1, 1),
                                                         datetime(period_end, 12, 31)))
    except TypeError:
        sst_atl3['time'] = sst_atl3.indexes['time'].to_datetimeindex()
        sst_atl3 = sst_atl3.sel(time=slice(datetime(period_str, 1, 1),
                                                         datetime(period_end, 12, 31)))   
    
    xdata_1 = np.arange(0,sst_atl3.shape[0],1)
    ydata_1_tmp = np.array(sst_atl3)
    
    
    sst_atl3 = sst_atl3.assign_coords(sst_dtd=('time',  nandetrend(ydata_1_tmp)))
    if sst_atl3.shape[0]>0:

        ssta_tmp,_ = ano_norm_t(sst_atl3.sst_dtd)
        amp_atl3_std = ssta_tmp.std(dim='time')
    else:
        amp_atl3_std = np.nan
        
    return amp_atl3_std

# Create the 31-year windows

In [6]:
year_start= np.arange(1900,2070,1)
year_end= np.arange(1930,2100,1)

period_str_cmip5_hist = 1900
period_end_cmip5_hist = 2014

period_str_cmip5_rcp45 = 2015
period_end_cmip5_rcp45 = 2099


x = x(s,m,t) with x representing the Atlantic Nino amplitude

X is the low-pass filtered values of Atlantic Nino amplitude, X$_{f}$ is the long-term trend and $\epsilon$ the internal long-term variability

X(s,m,t) = X$_{f}$(s,m,t) + $\epsilon$(s,m,t) hecne the dimension are s = 3, i.e. 3 scenarios, m =19, 19 models.

# Compute X(s,m,t)

In [10]:

## TOS ## 
amp_atl3_std = np.ones((len(scenario_CMIP6),len(model_list_CMIP6),year_start.shape[0]))*np.nan
for j in range(len(scenario_CMIP6)): ## loop on the scenarios
        print(scenario_CMIP6[j])
        for i in range(len(model_list_CMIP6)): ## loop on the models
            print(model_list_CMIP6[i])
            ## Take the historical data ##
            print('ts_Amon_'+model_list_CMIP6[i]+'_historical_r1i1p1f1_r144x72.ncts_Amon_'+model_list_CMIP6[i]+'_historical_r1i1p1f1_r144x72.nc')
            data_hist = xr.open_dataset(
                data_cmip_hist_ts+'ts_Amon_'+model_list_CMIP6[i]+'_historical_r1i1p1f1_r144x72.ncts_Amon_'+model_list_CMIP6[i]+'_historical_r1i1p1f1_r144x72.nc',
                decode_times=False)
            data_hist = data_hist.ts[:]
            time = pd.date_range('1850-01-15', freq='M', periods=data_hist.shape[0])
            data_hist['time'] = time
            data_hist_new = data_hist.sel(time=slice(datetime(period_str_cmip5_hist, 1, 1),
                                                         datetime(period_end_cmip5_hist, 12, 31)))

            ## Take the SSP245 data ##
            if scenario_CMIP6[j]=='ssp245': 
                print('ts_Amon_'+model_list_CMIP6[i]+'_ssp245_r1i1p1f1_r144x72.ncts_Amon_'+model_list_CMIP6[i]+'_ssp245_r1i1p1f1_r144x72.nc')
                data_rcp = xr.open_dataset(data_cmip_ssp245_ts+'ts_Amon_'+model_list_CMIP6[i]+'_ssp245_r1i1p1f1_r144x72.ncts_Amon_'+model_list_CMIP6[i]+'_ssp245_r1i1p1f1_r144x72.nc',
                decode_times=False)
                data_rcp = data_rcp.ts[:1020]
                time = pd.date_range('2015-01-15', freq='M', periods=data_rcp.shape[0])
                data_rcp['time'] = time
                data_rcp_new = data_rcp.sel(time=slice(datetime(period_str_cmip5_rcp45, 1, 1),
                                                         datetime(period_end_cmip5_rcp45, 12, 31)))
            ## Take the SSP585 data ##    
            else:
                print('ts_Amon_'+model_list_CMIP6[i]+'_ssp585_r1i1p1f1_r144x72.ncts_Amon_'+model_list_CMIP6[i]+'_ssp585_r1i1p1f1_r144x72.nc')
                data_rcp = xr.open_dataset(data_cmip_ssp585_ts+'ts_Amon_'+model_list_CMIP6[i]+'_ssp585_r1i1p1f1_r144x72.ncts_Amon_'+model_list_CMIP6[i]+'_ssp585_r1i1p1f1_r144x72.nc',
                decode_times=False)
                data_rcp = data_rcp.ts[:1020]
                time = pd.date_range('2015-01-15', freq='M', periods=data_rcp.shape[0])
                data_rcp['time'] = time
                data_rcp_new = data_rcp.sel(time=slice(datetime(period_str_cmip5_rcp45, 1, 1),
                                                         datetime(period_end_cmip5_rcp45, 12, 31)))
                
                
            ## Average over the ATL3 box ##
            sst_hist_atl3 = make_sst_atl3(data_hist_new)
            sst_rcp_atl3 = make_sst_atl3(data_rcp_new)
            
            ## Concatenate the historical and the scenario runs
            sst_all_atl3 = xr.concat([sst_hist_atl3,sst_rcp_atl3],dim='time')
            
            ## Compute the 31- year running standard deviation of the ATL3-averaged SST anomalies ##
            print('start running mean')
            for k in range(170):
                #print(j)
                amp_atl3_std[j,i,k] = compute_atl3_amplitude(sst_all_atl3,year_start[k],year_end[k])
            print(' ')

ssp245
ACCESS-CM2
ts_Amon_ACCESS-CM2_historical_r1i1p1f1_r144x72.ncts_Amon_ACCESS-CM2_historical_r1i1p1f1_r144x72.nc
ts_Amon_ACCESS-CM2_ssp245_r1i1p1f1_r144x72.ncts_Amon_ACCESS-CM2_ssp245_r1i1p1f1_r144x72.nc
start running mean
 
ACCESS-ESM1-5
ts_Amon_ACCESS-ESM1-5_historical_r1i1p1f1_r144x72.ncts_Amon_ACCESS-ESM1-5_historical_r1i1p1f1_r144x72.nc
ts_Amon_ACCESS-ESM1-5_ssp245_r1i1p1f1_r144x72.ncts_Amon_ACCESS-ESM1-5_ssp245_r1i1p1f1_r144x72.nc
start running mean
 
AWI-CM-1-1-MR
ts_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_r144x72.ncts_Amon_AWI-CM-1-1-MR_historical_r1i1p1f1_r144x72.nc
ts_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_r144x72.ncts_Amon_AWI-CM-1-1-MR_ssp245_r1i1p1f1_r144x72.nc
start running mean
 
BCC-CSM2-MR
ts_Amon_BCC-CSM2-MR_historical_r1i1p1f1_r144x72.ncts_Amon_BCC-CSM2-MR_historical_r1i1p1f1_r144x72.nc
ts_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_r144x72.ncts_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_r144x72.nc
start running mean
 
CAMS-CSM1-0
ts_Amon_CAMS-CSM1-0_historical_r1i1p1f1_r144x72.ncts_Amon_

# save the data into a netcdf file

In [11]:

scenario_CMIP6_new=['ssp245','ssp585']
time_new = pd.date_range('1915-06-01',freq='Y',periods=170)
running_mean_atl3_tmp = xr.Dataset({'amp_ATL3': (['scenario','model','time'], amp_atl3_std),
                               },
                      coords={'scenario': np.array(scenario_CMIP6_new),
                              'model': np.array(model_list_CMIP6),
                              'time':time_new} )

running_mean_atl3_tmp.to_netcdf(path_data_out+'ATL3_amplitude_CMIP6.nc')